My friends and I have a project to rate all of King Gizzard and the Lizard Wizard's albums and rank them. Here I want to take my notes and output them as a csv, so that I can add them to my friend's db. Mainly, I am practicing my Python string processing and Pandas. 

In [103]:
import pandas as pd
import numpy as np

## Make the dataframe

let's copy over my gizzard ratings from my notes

In [104]:
gizzards = '''Nonagon Infinity: 4 to 4.5
IDPLML: 3.5 to 4
Willouby's beach: 3
12 Bar Bruise: 4 to 4.5
butterfly: 3.5 to 4
Sketches of Brunswick East: 4 to 4.5
Gumboot Soup: 3.5 to 4
I'm in Your Mind Fuzz: 4 to 5
Fishing for Fishies: 3 to 3.5
Quarters!: 3.5 to 4.4
KL: 3
LW: 4
Petrodragonic: 4.5 to 5
Flying Microtonal banana: 4.5 to 5
Silver chord: 3.5 to 4.5
Laminated Denim: 3.5?
Polygondwanaland: 4.5 to 5
Chunky Shrapnel: 4.5 to 5
Live in SF: 4.5 to 5
Infest the Rat's Nest: 4 to 5
Omnium G: 3.5
Eyes Like the Sky: 1
Float Along - Fill Your Lungs: 2.5
Oddments:  1.5 to 2
Paper Mache Dream Balloon: 2.5
Murder of the Universe: 2.5
Made in Timeland: 2
Changes: 2
Flight B741: 2
Phantom Island: 2'''


In [105]:
def ratings_to_dict(notes: str):
    # GOAL: convert the notes on albums and rankings to a dictionary
    # The output will be a dictionary with the album titles as the keys,
    # and the full rating field as the values.
    #INPUT: notes are notes on the albums, in the general format
    #album title: rating \n
    #STRATEGY: I can split the strings on the new lines and the colon, then append 
    #them to the dict. I'm not planning on doing any error handling, or checking for misformatted entries.
    #SOLUTION
    albums = {}
    album_list = notes.split('\n')

    #go through the list and make the dictionary
    for album in album_list:
        key, value = album.split(':')
        albums[key] = value.lstrip() #remove the extra space in front of the rating. 
    return albums

albums = ratings_to_dict(gizzards)
albums

{'Nonagon Infinity': '4 to 4.5',
 'IDPLML': '3.5 to 4',
 "Willouby's beach": '3',
 '12 Bar Bruise': '4 to 4.5',
 'butterfly': '3.5 to 4',
 'Sketches of Brunswick East': '4 to 4.5',
 'Gumboot Soup': '3.5 to 4',
 "I'm in Your Mind Fuzz": '4 to 5',
 'Fishing for Fishies': '3 to 3.5',
 'Quarters!': '3.5 to 4.4',
 'KL': '3',
 'LW': '4',
 'Petrodragonic': '4.5 to 5',
 'Flying Microtonal banana': '4.5 to 5',
 'Silver chord': '3.5 to 4.5',
 'Laminated Denim': '3.5?',
 'Polygondwanaland': '4.5 to 5',
 'Chunky Shrapnel': '4.5 to 5',
 'Live in SF': '4.5 to 5',
 "Infest the Rat's Nest": '4 to 5',
 'Omnium G': '3.5',
 'Eyes Like the Sky': '1',
 'Float Along - Fill Your Lungs': '2.5',
 'Oddments': '1.5 to 2',
 'Paper Mache Dream Balloon': '2.5',
 'Murder of the Universe': '2.5',
 'Made in Timeland': '2',
 'Changes': '2',
 'Flight B741': '2',
 'Phantom Island': '2'}

Now we can use pandas methods to make the dataframe. We can tidy it up in a couple of lines

In [106]:
albums_df = pd.DataFrame(pd.Series(albums)).reset_index()
albums_df.columns = ['title', 'rating']
albums_df.sort_values(by = ['rating', 'title'], inplace=True, ascending =[False, True])
albums_df.reset_index(drop=True, inplace=True)
albums_df.head()

,title,rating
0,Chunky Shrapnel,4.5 to 5
1,Flying Microtonal banana,4.5 to 5
2,Live in SF,4.5 to 5
3,Petrodragonic,4.5 to 5
4,Polygondwanaland,4.5 to 5


Now I'd like to just take the first number of the rating to use as the rating, as I don't want to overrate them just because I've been obsessed with them lately.

Let's also change the index to match at ranking that starts at 1

In [107]:
albums_df['rating'] = albums_df['rating'].apply(lambda x: x.split(' to ')[0].removesuffix('?'))
albums_df.index+=1 #the index will be the ranking
albums_df.index.title = 'ranking'
albums_df

,title,rating
1,Chunky Shrapnel,4.5
2,Flying Microtonal banana,4.5
3,Live in SF,4.5
4,Petrodragonic,4.5
5,Polygondwanaland,4.5
6,I'm in Your Mind Fuzz,4
7,Infest the Rat's Nest,4
8,12 Bar Bruise,4
9,Nonagon Infinity,4
10,Sketches of Brunswick East,4


Next, I need some functions to be able to move albums up and down in this table, which means changing the index values and resorting by index. 

I'll make one function to move albums up the specified number of spaces, and one to move them down. 

In [108]:
def change_rank(title: str, number:int):
    '''GOAL: change the title albums ranking by the number
    output a new index for album_df that corresponds to the new ranking
     '''
    #INPUT: title is the album title we wish to move up, and number is the number of spots to move it up in the albums_df dataframe
    '''STRATEGY: We can use Pandas methods to change the indices. 
    1. Find the new rank value
    2. change the corresponding rank values in a numpy array. We need to change the value of the rank of title and any other albums
    that were leapfrogged
    '''
    #SOLUTION
    #Construct the new index
    new_ind = np.arange(len(albums_df))+1
    if number == 0: #bypass the rest and avoid any difficulties
       return new_ind
    
    #find the new rank
    #let's make this be case independent, to help with typos later
    title_rank = albums_df.loc[albums_df.title.str.lower() == title.lower(), :].index[0]
    #adding a positive value to the rank means decreasing it, counterintuitively
    new_rank = title_rank-number

    #check for mistakes in the number
    assert new_rank >= 1 and new_rank >= new_ind[-1]

    #construct the new ranking array and change the rank for title
   
    new_ind[title_rank-1] = new_rank

    #change the leapfrogged indices
    #The fun thing is that this can be done in one line for both signs of number
    new_ind[new_rank-1:title_rank-1:np.sign(number)]+=np.sign(number)
    
    return new_ind

## Change my rankings
Now we're ready to change the rankings of the albums to match my preferences